In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy
from skimage.color import rgb2gray
from skimage import morphology
from os import listdir, makedirs
from os.path import isfile, join, dirname, realpath, isdir
#np.set_printoptions(threshold=np.inf)


def binary_mask(im):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    abcd = cv2.Canny(gray,10,50)
    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1)) 
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)) 
    abcd = cv2.morphologyEx(np.float32(abcd), cv2.MORPH_CLOSE, ker1)
    abcd = cv2.morphologyEx(np.float32(abcd), cv2.MORPH_CLOSE, ker2)
    abcd = abcd>0
    abcd = morphology.remove_small_objects(abcd, min_size=150)
    for i in range(abcd.shape[0]):
        for j in range(abcd.shape[1]):
            if abcd[i,j] == True and gray[i,j] > 220 and gray[i,j] < 160:
                abcd[i,j] = False
    abcd = morphology.remove_small_objects(abcd, min_size=150)
    #mask = mask.astype(np.uint8)
    abcd = np.pad(abcd, ((0, 2),(0,2)), 'constant', constant_values=0)
    global U,D,L,R
    global coords
    U = D = L = R = 0
    coords = np.zeros((0,2) , dtype = int)
    clone = np.zeros((abcd.shape[0] , abcd.shape[1] , 3) , dtype = int)
    clone[: , : ,0] = abcd
    cy = gray.shape[0]
    cx = gray.shape[1]
    UD = 0
    LR = 0
    bound = np.zeros((0,4), dtype = int)
    bou = np.zeros((1,4),dtype = int)
    for j in range(cx):
        for i in range(cy):
            if (j<10 or j>(cx-10)) and clone[i,j,0]==1:
                oo=0
                for k in range(1,10):
                    if clone[i+k,j,0]==0:
                        oo=1
                        break
                if oo==0:
                    clone[:,:,0] = remove_blob(clone[:,:,0] , j,i)
            if clone[i,j,0] ==1 and not clone[i,j,1]==1:
                coords = np.vstack((coords,[i,j]))
                clone[i,j,1] = 1
                bou[0,0] = j
                bou[0,1],bou[0,2], bou[0,3] = orie(clone , i , j , j,i,i)
                UDLR = U+D+L+R
                UU = (U/UDLR)*100
                DD = (D/UDLR)*100
                LL = (L/UDLR)*100
                RR = (R/UDLR)*100
                print(UU,DD,LL,RR)
                #print(U + ' ' + D + ' ' + L + ' ' + R)
                UD = U-D
                UU = UU-DD
                LL = LL-RR
                LR = L-R
                print(UU,LL)
                if (UU<=15 and UU>= -15) or (UD<=35 and UD>= -35):
                    if (LL<= 15 and LL>= -15) or (LR<=35 and LR>= -35):
                        for o in range(coords.shape[0]):
                            y = coords[o,0]
                            x = coords[o,1]
                            clone[y,x,0] = 0
                    else:
                        bound = np.vstack((bound,bou))
                else:
                    bound = np.vstack((bound,bou))
                U = D = L = R = 0
                coords = np.zeros((0,2) , dtype = int)
                
    abcd = clone[:,:,0]
    abcdfill = scipy.ndimage.morphology.binary_fill_holes(abcd)
    abcdfill = abcdfill.astype(np.uint8)
    abcd = abcd.astype(np.uint8)
    new = abcdfill[:,:] - abcd[:,:]
    for k in range(bound.shape[0]):
        num = 0
        abcdd = abcd[(bound[k,2]):(bound[k,3]) , (bound[k,0]):(bound[k,1])]
        contours, hierarchy = cv2.findContours(abcdd, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if not contours==[]:
            cnt = contours[0]
            rect = cv2.minAreaRect(cnt)
            w,h = rect[1]
            box = cv2.boxPoints(rect)
            for x in range(bound[k,0],bound[k,1]):
                for y in range(bound[k,2],bound[k,3]):
                    if new[y,x] ==1:
                        yy = y
                        xx = x
                        num = num+1
            if num>3:
                if w<h:
                    if h < 3*w:
                        abcdfill = remove_blob(abcdfill,xx,yy )
                else:
                    if w < 3*h:
                        abcdfill = remove_blob(abcdfill,xx,yy )
    abcdfill = abcdfill>0
    abcdfill = morphology.remove_small_objects(abcdfill, min_size=150,connectivity=1)
    return abcdfill


def remove_blob(first,cx,cy):
    for y in range(cy-1,cy+2):
        for x in range(cx-1,cx+2):
            if first[y,x]==1:
                first[y,x] = 0
                first = remove_blob(first,x,y)
    return first



def orie(cloonie , i, j,xmax,ymin,ymax):
    global U,D,L,R,coords
    if xmax<j:
        xmax = j
    if i<ymin:
        ymin = i
    if i>ymax:
        ymax = i
    for y in range(i-1 , i+2):
        for x in range(j-1, j+2):
            if cloonie[y,x , 0]==1 and not cloonie[y,x,1]==1:
                if y>i:
                    D = D+1
                    if x>j:
                        R = R+1
                    elif x<j:
                        L = L+1
                elif y==i:
                    if x>j:
                        R = R+1
                    elif x<j:
                        L = L+1
                else:
                    U = U+1
                    if x>j:
                        R = R+1
                    elif x<j:
                        L = L+1
                cloonie[y,x,1]=1
                coords = np.vstack((coords,[y,x]))
                xmax , ymin, ymax = orie(cloonie, y,x , xmax,ymin,ymax)
    return xmax , ymin , ymax




clean
